In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
# Load the dataset
df = pd.read_csv('GOT_Transform_Batch2_v1.csv',low_memory=False)
df2=df.loc[:, df.columns.drop(['TRNSCR_DESCR', 'MUET','IELTS','NewID'])]
objList2 = df2.select_dtypes(include = "object").columns
objList3 = df2.select_dtypes(include = "float64").columns
df2[objList3].isnull().sum()
df2[objList3] = df2[objList3].fillna(0)
df2[objList3].isnull().sum()
objList2 = objList2.drop(['GOT2'])
le = LabelEncoder()
for feat in objList2:
    df2[feat] = le.fit_transform(df2[feat].astype(str))

df_drop_unuse = df2.drop(['MAX_STRM_id', 'PROG_ACTION', 'STATUS_DT', 'END_DT', 'CNT_DISC','INFO1','Prog_Length(Trimester)','TOT_CUMULATIVE','N_FINAL_RSLT_DESCR','N_HONOUR_DESCR'], axis=1)
df_drop_unuse['PROG_STATUS'].value_counts()
NewProgStatus = df_drop_unuse
NewProgStatus = NewProgStatus.drop("GOT2", axis=1)

In [8]:
# Separate the target variable and features
X = NewProgStatus.drop("PROG_STATUS", axis=1)
y = NewProgStatus.PROG_STATUS
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import xgboost as xgb



# Univariate feature selection - chi-squared test
chi_selector = SelectKBest(chi2, k=5)
X_train_chi = chi_selector.fit_transform(X_train, y_train)
X_test_chi = chi_selector.transform(X_test)

In [25]:
# Univariate feature selection - ANOVA F-test
f_selector = SelectKBest(f_classif, k=5)
X_train_f = f_selector.fit_transform(X_train, y_train)
X_test_f = f_selector.transform(X_test)

# Univariate feature selection - mutual information
mi_selector = SelectKBest(mutual_info_classif, k=5)
X_train_mi = mi_selector.fit_transform(X_train, y_train)
X_test_mi = mi_selector.transform(X_test)

In [26]:
# Regularization - L1 regularization
l1_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear'))
X_train_l1 = l1_selector.fit_transform(X_train, y_train)
X_test_l1 = l1_selector.transform(X_test)

# Regularization - L2 regularization
l2_selector = SelectFromModel(LogisticRegression(penalty="l2"))
X_train_l2 = l2_selector.fit_transform(X_train, y_train)
X_test_l2 = l2_selector.transform(X_test)

# Principal component analysis
pca_selector = PCA(n_components=5)
X_train_pca = pca_selector.fit_transform(X_train)
X_test_pca = pca_selector.transform(X_test)

c:\Users\about\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
from sklearn.metrics import accuracy_score

# Train and evaluate XGBoost model on each of the selected feature sets
models = {'Chi-squared test': XGBClassifier().fit(X_train_chi, y_train),
          'ANOVA F-test': XGBClassifier().fit(X_train_f, y_train),
          'Mutual information': XGBClassifier().fit(X_train_mi, y_train),
          'L1 regularization': XGBClassifier().fit(X_train_l1, y_train),
          'L2 regularization': XGBClassifier().fit(X_train_l2, y_train),
          'Principal component analysis': XGBClassifier().fit(X_train_pca, y_train)}

# Generate predictions for test set using each model

y_pred_chi = models['Chi-squared test'].predict(X_test_chi)
y_pred_f = models['ANOVA F-test'].predict(X_test_f)
y_pred_mi = models['Mutual information'].predict(X_test_mi)

y_pred_l1 = models['L1 regularization'].predict(X_test_l1)
y_pred_l2 = models['L2 regularization'].predict(X_test_l2)
y_pred_pca = models['Principal component analysis'].predict(X_test_pca)

# Evaluate performance of each model using accuracy metric

acc_chi = accuracy_score(y_test, y_pred_chi)
acc_f = accuracy_score(y_test, y_pred_f)
acc_mi = accuracy_score(y_test, y_pred_mi)

acc_l1 = accuracy_score(y_test, y_pred_l1)
acc_l2 = accuracy_score(y_test, y_pred_l2)
acc_pca = accuracy_score(y_test, y_pred_pca)

# Print accuracy scores for each model
print('Accuracy scores:')
print(f'Chi-squared test: {acc_chi}')
print(f'ANOVA F-test: {acc_f}')
print(f'Mutual information: {acc_mi}')
print(f'L1 regularization: {acc_l1}')
print(f'L2 regularization: {acc_l2}')
print(f'Principal component analysis: {acc_pca}')

Accuracy scores:
Chi-squared test: 0.7905282331511839
ANOVA F-test: 0.7741347905282332
Mutual information: 0.8697632058287796
L1 regularization: 0.9225865209471766
L2 regularization: 0.9061930783242259
Principal component analysis: 0.8633879781420765


In [30]:
import lightgbm as lgb

# Train and evaluate LightGBM model on each of the selected feature sets
models_lgb = {'Chi-squared test': lgb.LGBMClassifier().fit(X_train_chi, y_train),
              'ANOVA F-test': lgb.LGBMClassifier().fit(X_train_f, y_train),
              'Mutual information': lgb.LGBMClassifier().fit(X_train_mi, y_train),
              'L1 regularization': lgb.LGBMClassifier().fit(X_train_l1, y_train),
              'L2 regularization': lgb.LGBMClassifier().fit(X_train_l2, y_train),
              'Principal component analysis': lgb.LGBMClassifier().fit(X_train_pca, y_train)}



# Evaluate each model on test set
for name, model in models_lgb.items():
    if name == 'Chi-squared test':
        X_test_selected = X_test[chi_selector.get_support()]
    elif name == 'ANOVA F-test':
        X_test_selected = X_test[f_selector.get_support()]
    elif name == 'Mutual information':
        X_test_selected = X_test[mi_selector.get_support()]
    elif name == 'L1 regularization':
        X_test_selected = X_test.loc[:, l1_mask]
    elif name == 'L2 regularization':
        X_test_selected = X_test.loc[:, l2_mask]
    elif name == 'Recursive feature elimination':
        X_test_selected = rfecv.transform(X_test)
    elif name == 'Principal component analysis':
        X_test_selected = pca.transform(X_test)
    else:
        X_test_selected = X_test

    y_pred = model.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name} accuracy: {accuracy:.4f}')

ValueError: Item wrong length 142 instead of 1098.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import xgboost as xgb

# Correlation analysis
corr_matrix = X_train.corr()
corr_features = corr_matrix.index[abs(corr_matrix['at_risk']) >= 0.3]
X_train_corr = X_train[corr_features]
X_test_corr = X_test[corr_features]

# Univariate feature selection - chi-squared test
chi_selector = SelectKBest(chi2, k=5)
X_train_chi = chi_selector.fit_transform(X_train, y_train)
X_test_chi = chi_selector.transform(X_test)

# Univariate feature selection - ANOVA F-test
f_selector = SelectKBest(f_classif, k=5)
X_train_f = f_selector.fit_transform(X_train, y_train)
X_test_f = f_selector.transform(X_test)

# Univariate feature selection - mutual information
mi_selector = SelectKBest(mutual_info_classif, k=5)
X_train_mi = mi_selector.fit_transform(X_train, y_train)
X_test_mi = mi_selector.transform(X_test)

# Recursive feature elimination
rfecv_selector = RFECV(estimator=LogisticRegression(), step=1, cv=5, scoring='accuracy')
X_train_rfecv = rfecv_selector.fit_transform(X_train, y_train)
X_test_rfecv = rfecv_selector.transform(X_test)

# Regularization - L1 regularization
l1_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear'))
X_train_l1 = l1_selector.fit_transform(X_train, y_train)
X_test_l1 = l1_selector.transform(X_test)

# Regularization - L2 regularization
l2_selector = SelectFromModel(LogisticRegression(penalty="l2"))
X_train_l2 = l2_selector.fit_transform(X_train, y_train)
X_test_l2 = l2_selector.transform(X_test)

# Principal component analysis
pca_selector = PCA(n_components=5)
X_train_pca = pca_selector.fit_transform(X_train)
X_test_pca = pca_selector.transform(X_test)

# Train and evaluate XGBoost model on each of the selected feature sets
models = {'Correlation analysis': XGBClassifier().fit(X_train_corr, y_train),
          'Chi-squared test': XGBClassifier().fit(X_train_chi, y_train),
          'ANOVA F-test': XGBClassifier().fit(X_train_f, y_train),
          'Mutual information': XGBClassifier().fit(X_train_mi, y_train),
          'Recursive feature elimination': XGBClassifier().fit(X_train_rfecv, y_train),
          'L1 regularization': XGBClassifier().fit(X_train_l1, y_train),
          'L2 regularization': XGBClassifier().fit(X_train_l2, y_train),
          'Principal component analysis': XGBClassifier().fit(X_train_pca, y_train)}

# Evaluate each model on test set
for name, model in models.items():
    y_pred

In [10]:
# Univariate feature selection
univariate_feats = SelectKBest(f_classif, k=5).fit(X_train, y_train).get_support(indices=True)
mi_feats = SelectKBest(mutual_info_classif, k=5).fit(X_train, y_train).get_support(indices=True)

In [11]:
# Recursive feature elimination
rfe_feats = RFE(LogisticRegression(), n_features_to_select=5).fit(X_train, y_train).support_

c:\Users\about\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\about\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [12]:
# Regularization
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
clf = LogisticRegression(penalty='l1', solver='liblinear').fit(X_train_scaled, y_train)
l1_feats = np.nonzero(clf.coef_)[1]

In [13]:
# Principal component analysis
pca = PCA(n_components=5)
pca_feats = pca.fit_transform(X_train)

In [14]:
# Combine all selected features
selected_feats = list(set(univariate_feats) | set(mi_feats) | set(rfe_feats) | set(l1_feats))
selected_feats = list(set(selected_feats) | set(range(5)))  # add PCA features
selected_feats = list(set(selected_feats))  # remove duplicates

In [15]:
X_train_selected = X_train.iloc[:, selected_feats]
X_test_selected = X_test.iloc[:, selected_feats]
xgb = XGBClassifier()
xgb.fit(X_train_selected, y_train)
y_pred = xgb.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9207650273224044


In [6]:

# Correlation analysis
corr_matrix = X_train.corr()
corr_with_target = corr_matrix['target_variable'].abs().sort_values(ascending=False)
highly_corr_feats = corr_with_target[corr_with_target > 0.5].index.tolist()

# Univariate feature selection
univariate_feats = SelectKBest(f_classif, k=5).fit(X_train, y_train).get_support(indices=True)
mi_feats = SelectKBest(mutual_info_classif, k=5).fit(X_train, y_train).get_support(indices=True)

# Recursive feature elimination
rfe_feats = RFE(LogisticRegression(), n_features_to_select=5).fit(X_train, y_train).support_

# Regularization
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
clf = LogisticRegression(penalty='l1', solver='liblinear').fit(X_train_scaled, y_train)
l1_feats = np.nonzero(clf.coef_)[1]

# Principal component analysis
pca = PCA(n_components=5)
pca_feats = pca.fit_transform(X_train)

# Combine all selected features
selected_feats = highly_corr_feats + list(set(univariate_feats) | set(mi_feats) | set(rfe_feats) | set(l1_feats))
selected_feats = list(set(selected_feats) | set(range(5)))  # add PCA features
selected_feats = list(set(selected_feats))  # remove duplicates

# Train XGBoost model with selected features
X_train_selected = X_train.iloc[:, selected_feats]
X_test_selected = X_test.iloc[:, selected_feats]
xgb = XGBClassifier()
xgb.fit(X_train_selected, y_train)
y_pred = xgb.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

c:\Users\about\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\about\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

NameError: name 'highly_corr_feats' is not defined